# Klasyfikacja i ewaluacja

TODO: opis  -- oraz dlaczego używamy ewaluacji

In [48]:
import numpy as  np
from sklearn import datasets
from sklearn import tree
from sklearn import metrics

iris = datasets.load_iris()

In [49]:
print iris.data[:10]
print iris.target[:10]
print iris.target_names, '-->', set(iris.target)

[[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]
 [ 5.4  3.9  1.7  0.4]
 [ 4.6  3.4  1.4  0.3]
 [ 5.   3.4  1.5  0.2]
 [ 4.4  2.9  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]]
[0 0 0 0 0 0 0 0 0 0]
['setosa' 'versicolor' 'virginica'] --> set([0, 1, 2])


In [50]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()

clf.fit(iris.data, iris.target)

target_predicted = clf.predict(iris.data)

print metrics.classification.classification_report(iris.target, target_predicted, target_names=iris.target_names)

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        50
 versicolor       1.00      1.00      1.00        50
  virginica       1.00      1.00      1.00        50

avg / total       1.00      1.00      1.00       150



100 procent skuteczności? Wygląda na to że dokonaliśmy przeuczenia gdyż nasz model trenowaliśmy oraz testowaliśmy na tym samym zbiorze danych. Innymi słowy ocenialiśmy go nie sprawdzając jak dobrze model generalizuje.

Aby tego uniknąć podizelmy nasz zbiór na część testową oraz treningową w propocji 70-30.

In [51]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = \
            model_selection.train_test_split(iris.data, iris.target, test_size=0.3, random_state=0)
    
print X_train.shape[0], X_test.shape[0]

105 45


In [52]:
clf = tree.DecisionTreeClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print metrics.classification.classification_report(y_test, y_pred , target_names=iris.target_names)


             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        16
 versicolor       1.00      0.94      0.97        18
  virginica       0.92      1.00      0.96        11

avg / total       0.98      0.98      0.98        45



Z poprzedniego rozdizału z wykresu punktowego pamiętamy, że setosa była kwiatem, który znacznie różnył się od pozostałych dwu gatunków, a najwięcej problemów będzie sprawiało rozróżnienie virginicy i versicolor. To samo widać z tego rapot klasyfikacji, że najwięcej błędów dokonuje się w tych właśnie klasach. Z definicji tych miar precision i recall możemy wysnuć wniosek, że versicolor był często myony versicolor, Przy czym własnie 6% przypaków  versicolor było błędnia zaklasyfikowanych jako Virginica.

Aby zweryfikować, które klasy są ze soba mylone użyjmy Macierzy Konfuzji.

In [53]:
CM = metrics.classification.confusion_matrix(y_test, y_pred)

print CM

[[16  0  0]
 [ 0 17  1]
 [ 0  0 11]]


6% przypadków to tylko jeden egzemplarz danych, który zostal błędnie sklasyfikowany. Abt móc wykorzystać cały zbiór danych do ewaluacji przyjrzymu się koncepcji, która nazywa się walidacją krzyżową (cross-validation). 

In [54]:
kf = model_selection.KFold(n_splits=3)

for fold_no, (train_idx, test_idx) in enumerate(kf.split(X=iris.data, y = iris.target)):
    X_train = iris.data[train_idx]
    X_test = iris.data[test_idx]
    Y_train = iris.target[train_idx]
    Y_test = iris.target[test_idx]
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print metrics.classification.classification_report(Y_test, Y_pred, target_names=iris.target_names)

             precision    recall  f1-score   support

     setosa       0.00      0.00      0.00        50
 versicolor       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00        50

             precision    recall  f1-score   support

     setosa       0.00      0.00      0.00         0
 versicolor       0.00      0.00      0.00        50
  virginica       0.00      0.00      0.00         0

avg / total       0.00      0.00      0.00        50

             precision    recall  f1-score   support

     setosa       0.00      0.00      0.00         0
 versicolor       0.00      0.00      0.00        50

avg / total       0.00      0.00      0.00        50



Dlaczego wyszły nam wszędzie zera? podzieliiśmy zbiór na trzy podzbiory. Wyjściowo miał on 150 egzemplarzy a w każdej klasie było 50 egzemplarzy. W zwiazku z czym podzieliliśmy zbiór danych w taki sposób, że trenowaliśmy klasyfikator na dwu kwiatach a testowaliśmy go na trzecim. Nie daliśmy zatem żadnych szans w naszej ewaluacji w teście.

Moglibyśmy po prostu wymieszać zbiór wejściowy tak wybór kolejnych foldów wybierał losowe podzbiory. Ale wykorzystajmy tutaj Koncepcje losowania wartswowego (Stratified Sampling), które w przypadku niezbalansowanych danych zadbają o to aby prawodopobieństwo wybory danej klasy w zbiorze wejściowym było takie same w zbiorze testowym. Jako dodatkowy efekt cały zbiór danych będzie wybierany w losowej kolejności.

> Niezbalansowane zbiory danych (czyli takie, w których mamy znacząco różne wielkości w różnych klasach) wymagają specjalnego traktowania


In [55]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=3)


for fold_no, (train_idx, test_idx) in enumerate(kf.split(X=iris.data, y = iris.target)):
    X_train = iris.data[train_idx]
    X_test = iris.data[test_idx]
    Y_train = iris.target[train_idx]
    Y_test = iris.target[test_idx]
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print metrics.classification.classification_report(Y_test, Y_pred, target_names=iris.target_names)


             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        17
 versicolor       0.94      1.00      0.97        17
  virginica       1.00      0.94      0.97        17

avg / total       0.98      0.98      0.98        51

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        17
 versicolor       0.88      0.88      0.88        17
  virginica       0.88      0.88      0.88        17

avg / total       0.92      0.92      0.92        51

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        16
 versicolor       1.00      1.00      1.00        16
  virginica       1.00      1.00      1.00        16

avg / total       1.00      1.00      1.00        48



Oczywiście na powyższy kod istnieje funkcja pomocnicza

In [57]:
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score)
model_selection.cross_val_score(clf, iris.data, y=iris.target, cv=5, scoring=accuracy_scorer)

array([ 0.96666667,  0.96666667,  0.9       ,  0.96666667,  1.        ])

ImportError: No module named graphviz